In [1]:
from data import read_csv

0: ID00001
1: 0.515873589958172
2: 0.05
3: 0.644230769230769
4: C1
5: 4990
6: 1
7: A
8: M1
9: CNG
10: 60Nm@3500rpm
11: 40.36bhp@6000rpm
12: F8D Petrol Engine
13: 2
14: No
15: No
16: No
17: Yes
18: No
19: Drum
20: 796
21: 3
22: Manual
23: 5
24: Power
25: 4.6
26: 3445
27: 1515
28: 1475
29: 1185
30: No
31: No
32: No
33: No
34: No
35: No
36: No
37: Yes
38: No
39: No
40: No
41: Yes
42: 0
43: 0
